In [122]:
import os
import pandas as pd
from pathlib import Path
from pprint import pprint
from typing import List, Dict
from llama_index.core.schema import NodeWithScore
from sentence_transformers import SentenceTransformer, util
import torch
import hashlib
print("📂 Current working directory:", Path().resolve())
print("📄 Contents:", os.listdir())


📂 Current working directory: /Users/ywxiu/jasp-multimodal-rag/notebooks
📄 Contents: ['03_video_query.ipynb', '01_pdf_retrieval.ipynb', '02_video_loader.ipynb', '__pycache__', '01_pdf_loader_llamaindex.ipynb', '01_pdf_loader_page-level-summarization.ipynb', 'data']


In [123]:
import sys
from pathlib import Path

# add /Users/ywxiu/jasp-multimodal-rag/src to sys.path
project_root = Path.cwd().parent  # goes from notebooks → jasp-multimodal-rag
src_path = project_root / "src"
sys.path.append(str(src_path))

In [124]:

from retrieval.hybrid_retrieval import (
    build_vector_retriever,
    build_bm25_retriever,
    load_docs_for_bm25,
    rrf_fuse,
    rerank_cross_encoder,
    CHROMA_DIR,
    COLLECTION_NAME,
    EMBED_MODEL,
    CHUNKS_JSON,
    K_SEMANTIC,
    K_BM25,
    RRF_K,
    TOP_AFTER_RRF,
    TOP_FINAL,
)

print("✅ hybrid_retrieval imported successfully!")


✅ hybrid_retrieval imported successfully!


In [125]:

# 1️⃣ Choose your query
query = "How to split data files in JASP?"
CHROMA_DIR = "/Users/ywxiu/jasp-multimodal-rag/data/vector_store/chroma_db"


semantic_retriever = build_vector_retriever(
    persist_dir=CHROMA_DIR,
    collection_name=COLLECTION_NAME,
    embed_model_name=EMBED_MODEL,
)
sem_results = semantic_retriever.retrieve(query)

print(f"\nSemantic top-{K_SEMANTIC}: {len(sem_results)}\n{'='*60}")

for i, item in enumerate(sem_results, 1):
    node = item.node
    metadata = node.metadata
    text = node.text.strip().replace('\n', ' ')
    print(f"\n🔹 Result {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🎯 Score  : {item.score:.3f}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)



RuntimeError: MPS backend out of memory (MPS allocated: 9.06 GiB, other allocations: 640.00 KiB, max allowed: 9.07 GiB). Tried to allocate 16.00 MiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:




docs_for_bm25 = load_docs_for_bm25(CHUNKS_JSON)
bm25_retriever = build_bm25_retriever(docs_for_bm25)
kw_results = bm25_retriever.retrieve(query)

print(f"BM25 top-{K_BM25}: {len(kw_results)}")


2025-10-29 19:51:33.895 | INFO     | retrieval.hybrid_retrieval:load_docs_for_bm25:120 - BM25 corpus: 12 documents


2025-10-29 19:51:33.901 | INFO     | retrieval.hybrid_retrieval:build_bm25_retriever:168 - Initializing BM25 retriever with 12 documents...


BM25 top-10: 10


In [ ]:
print(f"\nBM25 top-{K_BM25}: {len(kw_results)}\n{'='*60}")

for i, item in enumerate(kw_results, 1):
    # BM25 retrievers often return Document-like objects
    metadata = getattr(item, "metadata", {})
    text = getattr(item, "text", str(item))  # fallback if structure differs

    print(f"\n🔹 Result {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)



BM25 top-10: 10

🔹 Result 1
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🧩 Chunk  : 1_0_0
   🗒️ Text   : 9    JASP

SPLITTING DATA FILES
If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced
for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group
to the Split box.

Variables             Descriptive Statis...
------------------------------------------------------------

🔹 Result 2
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🧩 Chunk  : 1_1_0
   🗒️ Text   : Deviation             6.424     7.040
                                    Skewness                   0.200    -0.176
                                    Std. Error of Skewness     0.137     0.110
                                    Kurtosis            -0.101          -0.397
Split                    ...
------------------------------------------------------------

🔹 Result 3
   📄 Source : test_pages25-28.pdf
   📑 Page   :

In [ ]:
fused = rrf_fuse(sem_results, kw_results, k=RRF_K, top_n=TOP_AFTER_RRF)
print(f"\n🔀 RRF shortlist: {len(fused)}\n{'='*60}")

for i, item in enumerate(fused, 1):
    node = getattr(item, "node", item)
    metadata = getattr(node, "metadata", {})
    text = getattr(node, "text", str(node))

    print(f"\n🔹 Fused Result {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🎯 Score  : {getattr(item, 'score', 'N/A')}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)



🔀 RRF shortlist: 10

🔹 Fused Result 1
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🎯 Score  : 5.365289211273193
   🧩 Chunk  : 1_0_0
   🗒️ Text   : 9    JASP

SPLITTING DATA FILES
If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced
for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group
to the Split box.

Variables             Descriptive Statis...
------------------------------------------------------------

🔹 Fused Result 2
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🎯 Score  : -3.707210063934326
   🧩 Chunk  : 1_1_0
   🗒️ Text   : Deviation             6.424     7.040
                                    Skewness                   0.200    -0.176
                                    Std. Error of Skewness     0.137     0.110
                                    Kurtosis            -0.101          -0.397
Split                    ...
---------------------------------------

In [ ]:
reranked = rerank_cross_encoder(fused, query, top_k=TOP_FINAL)
print(f"\n🏁 Reranked top-{TOP_FINAL}: {len(reranked)}\n{'='*60}")

for i, item in enumerate(reranked, 1):
    node = getattr(item, "node", item)
    metadata = getattr(node, "metadata", {})
    text = getattr(node, "text", str(node))

    print(f"\n⭐ Final Rank {i}")
    print(f"   📄 Source : {metadata.get('source', 'N/A')}")
    print(f"   📑 Page   : {metadata.get('page', 'N/A')}")
    print(f"   🔢 New Score : {getattr(item, 'score', 'N/A')}")
    print(f"   🧩 Chunk  : {metadata.get('chunk_id', 'N/A')}")
    print(f"   🗒️ Text   : {text[:300]}{'...' if len(text) > 300 else ''}")
    print("-"*60)

2025-10-29 19:57:55.950 | INFO     | retrieval.hybrid_retrieval:rerank_cross_encoder:224 - Cross-encoder rerank with SentenceTransformer: cross-encoder/ms-marco-MiniLM-L-6-v2



🏁 Reranked top-3: 3

⭐ Final Rank 1
   📄 Source : test_pages25-28.pdf
   📑 Page   : 1
   🔢 New Score : 5.365289211273193
   🧩 Chunk  : 1_0_0
   🗒️ Text   : 9    JASP

SPLITTING DATA FILES
If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced
for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group
to the Split box.

Variables             Descriptive Statis...
------------------------------------------------------------

⭐ Final Rank 2
   📄 Source : test_pages25-28.pdf
   📑 Page   : 2
   🔢 New Score : 2.429915428161621
   🧩 Chunk  : 2_4_1
   🗒️ Text   : (4) The image presents a scatterplot matrix derived from the JASP statistical software. Each point in the plot represents an individual data entry, and each cell within the grid is filled with a circle representing one of these entries. The size of the circles varies based on their position along th...
--------------------------------------

In [ ]:


# 3️⃣ Helper to format text
def short(txt, n=300):
    t = txt.strip().replace("\n", " ")
    return t[:n] + ("..." if len(t) > n else "")

def extract_info(nodes):
    data = []
    for i, n in enumerate(nodes, start=1):
        md = n.node.metadata or {}
        src = md.get("source") or md.get("file") or md.get("doc_id") or "unknown"
        page = md.get("page")
        tag = f"{src}" + (f":p{page}" if page is not None else "")
        data.append({
            "rank": i,
            "source": tag,
            "score": getattr(n, "score", None),
            "text": short(n.node.get_content()),
        })
    return data

# 4️⃣ Convert all results to DataFrames
df_sem = pd.DataFrame(extract_info(sem_results))
df_bm25 = pd.DataFrame(extract_info(kw_results))
df_rrf = pd.DataFrame(extract_info(fused))
df_rerank = pd.DataFrame(extract_info(reranked))

# 5️⃣ Combine top results side by side
max_rows = max(len(df_sem), len(df_bm25), len(df_rerank))
df_combined = pd.DataFrame({
    "Semantic (BM25 embedding)": df_sem["text"].reindex(range(max_rows)),
    "BM25 (lexical)": df_bm25["text"].reindex(range(max_rows)),
    "RRF fused": df_rrf["text"].reindex(range(max_rows)),
    "Final reranked": df_rerank["text"].reindex(range(max_rows)),
})
display(df_combined.fillna(""))

# 6️⃣ Optional: save for later manual review
# df_combined.to_csv("../notebooks/retrieval_debug_output.csv", index=False)


,Semantic (BM25 embedding),BM25 (lexical),RRF fused,Final reranked
0,9 JASP SPLITTING DATA FILES If there is a ...,9 JASP SPLITTING DATA FILES If there is a ...,9 JASP SPLITTING DATA FILES If there is a ...,9 JASP SPLITTING DATA FILES If there is a ...
1,Deviation 6.424 7.040 ...,Deviation 6.424 7.040 ...,Deviation 6.424 7.040 ...,(4) The image presents a scatterplot matrix de...
2,"The variables include the mean value, standard...",9 JASP The Distribution plot is based on s...,9 JASP The Distribution plot is based on s...,Adding to this effect are two diagonal lines e...
3,(4) The image presents a scatterplot matrix de...,(4) The image presents a scatterplot matrix de...,(4) The image presents a scatterplot matrix de...,
4,9 JASP The Distribution plot is based on s...,9 JASP Q-Q Plot Variable 4 20 2 0 ...,"The variables include the mean value, standard...",
5,Adding to this effect are two diagonal lines e...,Adding to this effect are two diagonal lines e...,Adding to this effect are two diagonal lines e...,
6,9 JASP CUSTOMISABLE PLOTS ...,9 JASP CUSTOMISABLE PLOTS ...,9 JASP Q-Q Plot Variable 4 20 2 0 ...,
7,9 JASP Q-Q Plot Variable 4 20 2 0 ...,The boxplots visualise several statistics desc...,9 JASP CUSTOMISABLE PLOTS ...,
8,"It features three bars of varying heights, sym...","The variables include the mean value, standard...","It features three bars of varying heights, sym...",
9,All bars have horizontal lines extending from ...,"It features three bars of varying heights, sym...",The boxplots visualise several statistics desc...,


In [126]:
import subprocess
import json

def ask_ollama(prompt: str, model: str = "llama3") -> str:
    """
    Send a prompt to a local Ollama model and return its response.
    Requires Ollama to be installed and running locally.
    """
    # Run the ollama command
    result = subprocess.run(
        ["ollama", "run", model],
        input=prompt.encode("utf-8"),
        capture_output=True,
        text=True,
    )
    if result.returncode != 0:
        raise RuntimeError(f"Ollama error: {result.stderr}")
    return result.stdout.strip()

# Example usage:
response = ask_ollama("Explain Reciprocal Rank Fusion (RRF) in simple terms.", model="llama3")
print(response)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


AttributeError: 'bytes' object has no attribute 'encode'